In [1]:
# %%
from datasets import load_dataset
import os
os.getcwd() 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
from test import *
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from utils import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
import logging
import sys
import transformers
import torch
import time
import argparse
from tqdm import tqdm
import string
from RoBERTa import *
from transformers import RobertaTokenizer
from transformers import AutoTokenizer

# %%

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser("main")

parser.add_argument('--valid_num_points', type=int,             default = 8,              help='validation data number')
parser.add_argument('--train_num_points', type=int,             default = 80,              help='train data number')
parser.add_argument('--model_name',       type=str,             default = 'roberta-base',   help='model name')
parser.add_argument('--max_length',       type=int,             default=128,                help='max_length')
parser.add_argument('--batch_size',       type=int,             default=8,                  help='Batch size')
parser.add_argument('--num_workers',      type=int,             default=0,                  help='num_workers')
parser.add_argument('--epochs',           type=int,             default=5000,                  help='num of epochs')
parser.add_argument('--lr',               type=float,           default=1e-5,               help='lr')
parser.add_argument('--gamma',            type=float,           default=1,                  help='lr*gamma after each test')

args = parser.parse_args(args=[])#(args=['--batch_size', '8',  '--no_cuda'])#used in ipynb
print(args)

dataset = load_dataset('glue', 'mnli')

# %%
train = dataset['train'][:args.train_num_points]
valid = dataset['train'][-args.valid_num_points:]

tokenizer = RobertaTokenizer.from_pretrained(args.model_name)
#mnli
#The Multi-Genre Natural Language Inference Corpus is a crowdsourced collection of sentence pairs with textual entailment annotations. Given a premise sentence and a hypothesis sentence, the task is to predict whether the premise entails the hypothesis (entailment), contradicts the hypothesis (contradiction), or neither (neutral). The premise sentences are gathered from ten different sources, including transcribed speech, fiction, and government reports. The authors of the benchmark use the standard test set, for which they obtained private labels from the RTE authors, and evaluate on both the matched (in-domain) and mismatched (cross-domain) section. They also uses and recommend the SNLI corpus as 550k examples of auxiliary training data.

# %%
train_data = get_Dataset(train, tokenizer,max_length=args.max_length)
train_dataloader = DataLoader(train_data, sampler= SequentialSampler(train_data), 
                        batch_size=args.batch_size, pin_memory=args.num_workers>0, num_workers=args.num_workers)
valid_data = get_Dataset(valid, tokenizer,max_length=args.max_length)
valid_dataloader = DataLoader(valid_data, sampler=SequentialSampler(valid_data), 
                        batch_size=args.batch_size, pin_memory=args.num_workers>0, num_workers=args.num_workers)

# %%
model = TextClassifier(args).to(device)
model.train(train_dataloader,train_dataloader,device)
    

# %%





Namespace(batch_size=8, epochs=5000, gamma=1, lr=1e-05, max_length=128, model_name='roberta-base', num_workers=0, train_num_points=80, valid_num_points=8)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Epoch 1/5000, Train Loss: 1.1003
Validation Loss: 1.0979
Validation Accuracy: 0.3875
Epoch 2/5000, Train Loss: 1.0981
Validation Loss: 1.0959
Validation Accuracy: 0.5500
Epoch 3/5000, Train Loss: 1.0984
Validation Loss: 1.0941
Validation Accuracy: 0.5250
Epoch 4/5000, Train Loss: 1.0848
Validation Loss: 1.0881
Validation Accuracy: 0.3875
Epoch 5/5000, Train Loss: 1.0894
Validation Loss: 1.0782
Validation Accuracy: 0.4625
Epoch 6/5000, Train Loss: 1.0816
Validation Loss: 1.0513
Validation Accuracy: 0.5750
Epoch 7/5000, Train Loss: 1.0340
Validation Loss: 0.9812
Validation Accuracy: 0.6500
Epoch 8/5000, Train Loss: 0.9760
Validation Loss: 0.8935
Validation Accuracy: 0.6875
Epoch 9/5000, Train Loss: 0.9142
Validation Loss: 0.8312
Validation Accuracy: 0.7125
Epoch 10/5000, Train Loss: 0.8588
Validation Loss: 0.7671
Validation Accuracy: 0.7875
Epoch 11/5000, Train Loss: 0.8104
Validation Loss: 0.7015
Validation Accuracy: 0.9625
Epoch 12/5000, Train Loss: 0.7237
Validation Loss: 0.6418
Valid